In [ ]:
import os
import glob
from pathlib import Path
import yaml

import logging

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Show more columns
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)
# Initiate Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def get_directories():

    # Get Current Working Directory
    CWD = Path().resolve()

    # 
    # Priority: ENV → script → Jupyter fallback
    env_root = os.getenv("APP_ROOT")

    if env_root:
        APP_ROOT = Path(env_root).resolve()

    else:
        try:
            # Script Path
            APP_ROOT = Path(__file__).resolve().parents[1]
        except NameError:
            # Notebook execution path
            APP_ROOT = CWD.parent

    # Set Data Directory
    DATA_DIR = APP_ROOT / "data"

    # Log everything for debugging
    logger.info(f"CWD:             {CWD}")
    logger.info(f"APP_ROOT:        {APP_ROOT}")
    logger.info(f"DATA Directory:  {DATA_DIR}")

    return APP_ROOT, DATA_DIR

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

def load_data(DATA_PATH, DATA_FILE, **kwargs):

    """
        Load data file with
    """

    try:
        data = pd.read_csv(DATA_PATH/DATA_FILE, **kwargs)
        logger.info(f"Data File Loaded: {data.info()}")
        return data

    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

    
    #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 


In [ ]:
# Global Variables

# Folder Locations
APP_ROOT, DATA_DIR = get_directories()

RAW_DATA_DIR = DATA_DIR / "raw"
RAW_DATA_SUBDIR = "NASA_Turbofan_Jet_Engine_Dataset/CMaps"

RAW_DATA_PATH = RAW_DATA_DIR / RAW_DATA_SUBDIR

BRONZE_DATA_DIR = DATA_DIR / "bronze"
BRONZE_DATA_DIR = "turbofan_train_bronze.csv"


In [ ]:
print(RAW_DATA_PATH)

In [ ]:
# There are 26 columns in the dataset. 
# The first five columns have a different format then the remaining 21. 
# We will addresss these columns first then add in the remainingg 21. 

# Creating column name list for the first five. 
COLUMN_NAMES = [
    "unit",
    "cycle",
    "op_setting_1",
    "op_setting_2",
    "op_setting_3",
]

# As stated the remaining 21 columns share a similar name structure of sensor then a number
# So we will generate them using a loop and appended them to our list:
COLUMN_NAMES += [f"s{i}" for i in range(1, 22)]

# Display Column Names to verify success.
len(COLUMN_NAMES), COLUMN_NAMES[:10]


In [ ]:
# Load Data Import Keywords/Additional parameters.
data_import_parameters = {"sep":"\s+", "header":None, "names": COLUMN_NAMES}

# Anomaly window for when errors occur
ANOMALY_HORIZON = 30

FD_IDS = ["FD001", "FD002", "FD003", "FD004"]

# Creating a list 
all_training_dfs = []

for fd_id in FD_IDS:

    RAW_FD_PATH = RAW_DATA_PATH
    RAW_DATA_FILE = f"train_{fd_id}.txt"

    raw_df = load_data(RAW_FD_PATH, RAW_DATA_FILE, **data_import_parameters)

    raw_df.columns = COLUMN_NAMES

    max_cycle = raw_df.groupby("unit")["cycle"].max().rename("max_cycle")

    df = raw_df.merge(max_cycle, on="unit", how="left")

    df["RUL"] = df["max_cycle"] - df["cycle"]

    df = df.drop(columns=["max_cycle"])

    df["anomaly_flag"] = (df["RUL"] <= ANOMALY_HORIZON).astype(int)

    df["dataset_name"] = "TURBOFAN"
    df["fd_id"] = fd_id

    df = df.rename(columns={
                "unit": "machine_id",
                "cycle": "time_index",
                }, inplace=True)
    
    df["event_time"] = df["time_index"]

    core_columns = ["dataset_name", "fd_id", "machine_id", "time_index", "event_time"]
    label_columns = ["RUL", "anomaly_flag"]

    data_columns_list = list(df.columns)

    for column in core_columns + label_columns: 
        if column in data_columns_list:
            data_columns_list.remove(column)

    ordered_columns = core_columns + label_columns + data_columns_list

    df_ordered = df[ordered_columns].copy()

    all_training_dfs.append(df_ordered)


bronze_dfs = pd.concat(all_training_dfs, axis=0, ignore_index=True)

In [ ]:
bronze_dfs.shape

In [ ]:
bronze_dfs.info()

In [ ]:
bronze_dfs.head(10)

In [ ]:
out_path_all = (BRONZE_DATA_DIR) / "turbofan_all_train_bronze.csv"

bronze_dfs.to_csv(out_path_all, index=False)